In [2]:
from transformers import FastImageProcessor, FastForSceneTextRecognition, AutoConfig, AutoBackbone
from transformers import TextNetConfig
from transformers import TextNetBackbone
from PIL import Image
import requests

url = "https://huggingface.co/datasets/Raghavan/fast_model_samples/resolve/main/img657.jpg"
image = Image.open(requests.get(url, stream=True).raw).convert("RGB")
processor = FastImageProcessor.from_pretrained("jadechoghari/fast-tiny")


preprocessor_config.json: 100%|██████████| 523/523 [00:00<00:00, 3.03MB/s]


In [3]:
model = FastForSceneTextRecognition.from_pretrained("jadechoghari/fast-tiny")

config.json: 100%|██████████| 4.83k/4.83k [00:00<00:00, 11.1MB/s]
model.safetensors: 100%|█████████▉| 54.3M/54.3M [00:00<00:00, 87.8MB/s]


In [4]:
import torch
inputs = processor(image, return_tensors="pt")
# forward pass
with torch.no_grad():
    outputs = model(pixel_values=inputs["pixel_values"])

/home/user/app/transformers/src/transformers/models/fast/modeling_fast.py:294: UserWarning: `nn.functional.upsample` is deprecated. Use `nn.functional.interpolate` instead.
  return F.upsample(layer_out, size=(height, width), mode="bilinear")


In [23]:
target_sizes = [image.size]
target_sizes = [(300, 400)]
threshold = 0.88
text_locations = processor.post_process_text_detection(outputs, target_sizes, threshold, bbox_type="rect")

image size (640, 864)


In [21]:
target_sizes

[(300, 400)]

In [24]:
text_locations

[{'bboxes': [[151, 151, 160, 56, 355, 74, 346, 169],
   [181, 205, 328, 192, 332, 249, 186, 261]],
  'scores': [0.918617844581604, 0.9378566741943359]}]

In [8]:
import numpy as np
import cv2
import math

def generate_bbox(keys, label, score, scales, cfg):
    label_num = len(keys)
    bboxes = []
    scores = []
    for index in range(1, label_num):
        i = keys[index]
        ind = (label == i)
        ind_np = ind.data.cpu().numpy()
        points = np.array(np.where(ind_np)).transpose((1, 0))
        if points.shape[0] < cfg.test_cfg.min_area:
            label[ind] = 0
            continue
        score_i = score[ind].mean().item()
        if score_i < cfg.test_cfg.min_score:
            label[ind] = 0
            continue

        if cfg.test_cfg.bbox_type == 'rect':
            rect = cv2.minAreaRect(points[:, ::-1])
            alpha = math.sqrt(math.sqrt(points.shape[0] / (rect[1][0] * rect[1][1])))
            rect = (rect[0], (rect[1][0] * alpha, rect[1][1] * alpha), rect[2])
            bbox = cv2.boxPoints(rect) * scales

        elif cfg.test_cfg.bbox_type == 'poly':
            binary = np.zeros(label.shape, dtype='uint8')
            binary[ind_np] = 1
            contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            bbox = contours[0] * scales
        bbox = bbox.astype('int32')
        bboxes.append(bbox.reshape(-1).tolist())
        scores.append(score_i)
    return bboxes, scores

In [ ]:
generate_bbox()